In [11]:
from basketball_reference_web_scraper import client
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import lxml.html as lh
import numpy as np
import unidecode


AttributeError: module 'pandas' has no attribute 'compat'

In [10]:
sys.executable

'/Users/erichochberger/Desktop/Stat_301_03_Final_Project/.venv/bin/python3'

In [14]:
advanced_stats = client.players_advanced_season_totals(season_end_year=2019)

normal_stats = client.players_season_totals(season_end_year=2018)

testdf = pd.DataFrame(advanced_stats)

In [15]:
testdf['minutes_played_total'] = testdf.groupby('name').minutes_played.transform('sum')
testdf['proportion'] = testdf['minutes_played']/testdf['minutes_played_total']
num_cols = list(testdf.select_dtypes(include=['int', 'float64']))
unwanted_num_cols = ['age', 'minutes_played', 'games_played']
for col in unwanted_num_cols:
    num_cols.remove(col)
for col in num_cols:
    testdf[col] = testdf[col]*testdf['proportion']
testdf = testdf.groupby('name').agg('sum')

player_counts = pd.DataFrame(testdf.groupby('name').size())

In [39]:
def clean_advanced(year):
    advanced_stats = client.players_advanced_season_totals(season_end_year=year)
    df = pd.DataFrame(advanced_stats)
    # Handle quirk in data where traded players are represented as multiple observations
    df['minutes_played_total'] = df.groupby('name').minutes_played.transform('sum')
    df['proportion'] = df['minutes_played']/df['minutes_played_total']
    num_cols = list(df.select_dtypes(include=['int', 'float64']))
    unwanted_num_cols = ['age', 'minutes_played', 'games_played']
    for col in unwanted_num_cols:
        num_cols.remove(col)
    for col in num_cols:
        df[col] = df[col]*df['proportion']
    df_grouped = df.groupby('name')[num_cols].agg('sum')
    df_grouped['age'] = df.groupby('name')['age'].agg('mean')
    df_grouped['year'] = year
    return df_grouped
def clean_basic(year):
    basic_stats = client.players_season_totals(season_end_year=year)
    df = pd.DataFrame(basic_stats)
    df = df.groupby('name').agg('sum')
    num_cols = list(df.select_dtypes(include=['int', 'float64']))
    unwanted_num_cols = ['age', 'minutes_played', 'games_played', 'games_started']
    for col in unwanted_num_cols:
        num_cols.remove(col)
    for col in num_cols:
        df[col] = (df[col]/df['minutes_played'])*48
    df['year'] = year
    return df

def ScrapePage(url):
    r = requests.get(url)
    # Get number of pages
    soup = BeautifulSoup(r.content, features='html.parser')
    page_total = soup.find(class_="page-numbers").get_text()
    page_total = re.sub('.*of', '', page_total).strip()
    page_total = int(page_total)
    r = requests.get(url)
    doc = lh.fromstring(r.content)
    tr_elements = doc.xpath('//tr')
    # Create empty list
    col = []
    i = 0
    # For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i += 1
        name = t.text_content()
        col.append((name, []))
    for j in range(len(tr_elements)):
        # T is our j'th row
        T = tr_elements[j]
        # i is the index of our column
        i = 0
        # Iterate through each element of the row
        for t in T.iterchildren():
            data = t.text_content()
            # Check if row is empty
            if i > 0:
                # Convert any numerical value to integers
                try:
                    data = int(data)
                except:
                    pass
            # Append the data to the empty list of the i'th column
            col[i][1].append(data)
            # Increment i for the next column
            i += 1
    dict = ({title: column for (title, column) in col})
    return dict, page_total


def GetSalaries(year):
    df = []
    dict = {}
    url = 'http://www.espn.com/nba/salaries/_/year/' + year + '/'
    data, page_total = ScrapePage(url)
    df.append(pd.DataFrame(data))
#Get Salary Data
    for k in range(2,page_total + 1):
        url = 'http://www.espn.com/nba/salaries/_/year/' + year + '/page/' + str(k)
        Dict = ScrapePage(url)[0]
        df.append(pd.DataFrame(Dict))
    df = pd.concat(df)
    df['year'] = int(year)
    df = df[df['RK'] != "RK"]
    df.reset_index(inplace=True, drop=True)
    #Convert salary to numeric
    for i in range(len(df)):
        df.loc[i, 'name'] = re.sub(',.*', '', df['NAME'][i])
        df.loc[i, 'salary'] = re.sub('\$', '', df['SALARY'][i])
        df.loc[i, 'salary'] = re.sub(',', '', df['salary'][i])
 #Get Rid of text rows
        df['salary'] = pd.to_numeric(df['salary'])
    return df


In [17]:
#Get Player Statistics
all_stats_2011_2020 = []
for year in range(2011, 2020):
    df = clean_advanced(year)
    df1 = clean_basic(year)
    stats = pd.merge(df, df1, how='inner', left_on=['name', 'age'] right_on=['name', 'age'])
    all_stats_2011_2020.append(stats)
all_stats_2011_2020 = pd.concat(all_stats_2011_2020)
all_stats_2011_2020.isna().sum()

# b_stats_2011_2020 = []
# for year in range(2011, 2020):
#     df = clean_basic(year)
#     b_stats_2011_2020.append(df)
# b_stats_2011_2020 = pd.concat(b_stats_2011_2020).reset_index()

In [52]:
all_stats_2011_2020 = pd.concat([a_stats_2011_2020.reset_index(drop=True), b_stats_2011_2020.reset_index(drop=True)], axis=1)

In [40]:
#Get Salaries
salarydf = []
for year in range(2011, 2020):
    salarydf.append(GetSalaries(str(year)))


In [46]:
salarydf = pd.concat(salarydf)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [47]:
salarydf = salarydf.reset_index()

In [56]:
for i in range(len(salarydf)):
    salarydf['name'][i] = unidecode.unidecode(salarydf['name'][i])
    salarydf['name'][i] = re.sub('\*', '', salarydf['name'][i])
    salarydf['name'][i] = re.sub('\.', '', salarydf['name'][i])
    salarydf['name'][i] = salarydf['name'][i].replace("'", '')
    salarydf['name'][i] = salarydf['name'][i].replace(" ", '_').lower()


In [68]:
all_stats_2011_2020 = all_stats_2011_2020.reset_index()
for i in range(len(all_stats_2011_2020)):
    all_stats_2011_2020['name'][i] = str(all_stats_2011_2020['name'][i])
    all_stats_2011_2020['name'][i] = unidecode.unidecode(all_stats_2011_2020['name'][i])
    all_stats_2011_2020['name'][i] = re.sub('\*', '', all_stats_2011_2020['name'][i])
    all_stats_2011_2020['name'][i] = re.sub('\.', '', all_stats_2011_2020['name'][i])
    all_stats_2011_2020['name'][i] = all_stats_2011_2020['name'][i].replace("'", '')
    all_stats_2011_2020['name'][i] = all_stats_2011_2020['name'][i].replace(" ", '_').lower()

In [69]:
all_stats_2011_2020.head()

,level_0,index,name,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,...,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points,year
0,0,0,A.J. Price,10.700000,0.454000,0.466000,0.253000,2.300000,7.800000,5.000000,...,4.890566,0.966038,3.381132,6.701887,1.750943,0.060377,3.200000,3.683019,19.501887,2011
1,1,1,Aaron Brooks,13.136682,0.490050,0.399425,0.240155,1.757866,5.157944,3.489486,...,5.233645,0.747664,2.168224,8.710280,1.271028,0.112150,3.700935,4.299065,23.700935,2011
2,2,2,Aaron Gray,11.000000,0.566000,0.000000,0.343000,13.400000,25.800000,19.600000,...,3.073446,5.333333,10.124294,1.446328,0.994350,1.084746,2.892655,8.677966,11.661017,2011
3,3,3,Acie Law,9.888705,0.496264,0.197609,0.349284,1.884848,8.210744,4.997796,...,4.231405,0.793388,3.371901,5.553719,2.115702,0.000000,2.842975,4.033058,14.214876,2011
4,4,4,Al Harrington,12.300000,0.527000,0.486000,0.173000,5.400000,17.200000,11.500000,...,3.372973,2.190991,7.380180,2.911712,1.124324,0.288288,3.055856,5.823423,22.054054,2011
